In [9]:
import pandas as pd

# Reading the txt file into a pandas DataFrame
df = pd.read_csv('/media/hbai/data/code/LITE/intermidery_frame_trackid_confs_0.55.txt', sep=" ", header=None, names=['frame_id', 'track_id', 'confidence'])

# Display the DataFrame
df


,frame_id,track_id,confidence
0,2,0,0.90
1,2,1,0.89
2,2,6,0.82
3,2,3,0.89
4,2,5,0.85
...,...,...,...
354700,3314,809,0.39
354701,3314,807,0.27
354702,3314,756,0.62
354703,3314,732,0.34


In [6]:
!pip install nbformat --upgrade


  Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
  Using cached fastjsonschema-2.20.0-py3-none-any.whl (23 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl (88 kB)
  Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
  Using cached rpds_py-0.20.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (354 kB)
  Using cached referencing-0.35.1-py3-none-any.whl (26 kB)


In [5]:
import pandas as pd
import plotly.express as px


# Plot using Plotly
fig = px.scatter(df[df['track_id']<10], x='frame_id', y='confidence', color='track_id', title='Confidence Track ID=0 over Frames')

# Show the plot
fig.show()


In [10]:
import pandas as pd
import plotly.graph_objects as go
from scipy import stats

# Read the txt file into a pandas DataFr
# Aggregate confidence for each track_id
agg_df = df.groupby('track_id')['confidence'].agg(['mean', 'count', 'std']).reset_index()

# Calculate the standard error of the mean (SEM)
agg_df['sem'] = agg_df['std'] / agg_df['count'] ** 0.5

# Calculate the t-critical value for the 95% confidence interval
confidence_level = 0.95
degrees_freedom = agg_df['count'] - 1
t_critical = stats.t.ppf((1 + confidence_level) / 2, degrees_freedom)

# Handle cases where degrees of freedom is zero (count = 1)
t_critical[degrees_freedom == 0] = 0

# Calculate the margin of error
agg_df['margin_of_error'] = t_critical * agg_df['sem']

# Create a scatter plot with error bars
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=agg_df['track_id'],
    y=agg_df['mean'],
    error_y=dict(
        type='data',
        array=agg_df['margin_of_error'],
        visible=True
    ),
    mode='markers',
    marker=dict(size=10, color=agg_df['track_id'], colorscale='Viridis'),
    name='Mean Confidence'
))

# Update the layout for better readability
fig.update_layout(
    title='Mean Confidence per Track ID with 95% Confidence Interval',
    xaxis_title='Track ID',
    yaxis_title='Mean Confidence',
    showlegend=False
)

# Show the plot
fig.show()


# check correctness of confidence of tracks

In [128]:
# read pickle /media/hbai/data/code/LITE/results/scenarios/MOT20-train/LITEDeepSORT__input_1280__conf_.25/MOT20-01.pkl
import pickle
import pandas as pd
import numpy as np
pickle_file = '/media/hbai/data/code/LITE/results/scenarios/MOT20-train/LITEDeepSORT__input_1280__conf_.25/MOT20-01.pkl'
df_all_alpha = pd.read_pickle(pickle_file)
print(df_all_alpha.keys())
df_all_055 = pd.DataFrame(df_all_alpha[0.55], columns=['frame_id', 'indexes', 'track_id', 'confidence'])
df_all_055

dict_keys([0.05, 0.1, 0.15000000000000002, 0.2, 0.25, 0.3, 0.35000000000000003, 0.4, 0.45, 0.5, 0.55, 0.6000000000000001, 0.6500000000000001, 0.7000000000000001, 0.7500000000000001, 0.8, 0.8500000000000001, 0.9000000000000001, 0.9500000000000001])


,frame_id,indexes,track_id,confidence
0,2,0,1,0.90
1,2,1,2,0.89
2,2,6,7,0.82
3,2,3,4,0.89
4,2,5,6,0.85
...,...,...,...,...
9738,428,20,23,0.75
9739,428,9,10,0.70
9740,428,39,70,0.74
9741,428,41,74,0.63


In [129]:
df_all_alpha

array([  1,   2,   7,   4,   6,  12,   3,   8,   5,  13,   9,  10,  17,
        15,  22,  11,  26,  27,  23,  25,  32,  33,  21,  36,  16,  38,
        46,  47,  53,  56,  59,  63,  65,  70,  72,  74,  66,  76,  86,
        88,  75,  96,  98,  93,  85,  82, 108, 119, 118, 122, 123, 125,
       127, 132, 131, 147, 150, 155, 149, 158])

In [141]:
import pandas as pd
import plotly.graph_objects as go
from scipy import stats


def plot_confidence(df_all_alpha, alpha):
    # Create DataFrame for each alpha level
    df_alpha = df_all_alpha[alpha]
    df = pd.DataFrame(df_alpha, columns=['frame_id', 'indexes', 'track_id', 'confidence'])

    # Aggregate confidence for each track_id
    agg_df = df.groupby('track_id')['confidence'].agg(['mean', 'count', 'std']).reset_index()

    # Calculate the standard error of the mean (SEM)
    agg_df['sem'] = agg_df['std'] / agg_df['count'] ** 0.5

    # Calculate the t-critical value for the 95% confidence interval
    confidence_level = 0.95
    degrees_freedom = agg_df['count'] - 1
    t_critical = stats.t.ppf((1 + confidence_level) / 2, degrees_freedom)

    # Handle cases where degrees of freedom is zero (count = 1)
    t_critical[degrees_freedom == 0] = 0

    # Calculate the margin of error
    agg_df['margin_of_error'] = t_critical * agg_df['sem']

    # Create a scatter plot with error bars
    scatter_fig = go.Figure()

    scatter_fig.add_trace(go.Scatter(
        x=agg_df['track_id'],
        y=agg_df['mean'],
        error_y=dict(
            type='data',
            array=agg_df['margin_of_error'],
            visible=True
        ),
        mode='markers',
        marker=dict(size=10, color=agg_df['track_id'], colorscale='Viridis'),
        name=f'Scatter (Alpha {alpha})'
    ))

    scatter_fig.update_layout(
        title=f'Scatter: Mean Confidence per Track ID with 95% Confidence Interval (Alpha {alpha})',
        xaxis_title='Track ID',
        yaxis_title='Mean Confidence',
        showlegend=False
    )

    scatter_fig.show()

    # Create a boxplot for the same alpha level
    boxplot_fig = go.Figure()

    boxplot_fig.add_trace(go.Box(
        y=df['confidence'],
        name=f'Boxplot (Alpha {alpha})',
        marker_color='purple'
    ))

    boxplot_fig.update_layout(
        title=f'Boxplot of Confidence Scores (Alpha {alpha})',
        yaxis_title='Confidence',
        showlegend=False
    )

    boxplot_fig.show()

plot_confidence(df_all_alpha, .5)

In [108]:
from collections import defaultdict

# Sampl

# Get all unique values in the second column (index 1)
def get_unique_values_in_second_column(data):
    unique_values = set()
    for key, value_list in data.items():
        for tup in value_list:
            unique_values.add(tup[2])  # Add the second element of the tuple (index 1)
    return unique_values

# Get unique values in the second column
unique_values_in_second_column = get_unique_values_in_second_column(df_all_alpha)

print("Unique values in the second column:", unique_values_in_second_column)


Unique values in the second column: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 155, 15, 16, 17, 130, 19, 131, 21, 22, 23, 145, 25, 26, 27, 28, 132, 149, 150, 32, 33, 154, 35, 36, 156, 38, 158, 40, 46, 47, 53, 56, 59, 62, 63, 65, 66, 70, 72, 74, 75, 76, 82, 85, 86, 88, 93, 96, 98, 147, 105, 108, 116, 118, 119, 122, 123, 125, 127}


In [116]:
import pandas as pd

# Read the tracker predictions into a DataFrame
df_gt = pd.read_csv(
    '/media/hbai/data/code/LITE/results/scenarios/MOT20-train/LITEDeepSORT__input_1280__conf_.25/data/MOT20-01.txt', 
    header=None, 
    names=[
        'frame_id', 'track_id', 'x', 'y', 'width', 'height', 
        'confidence', 'class_id', 'visibility', 'attr1', 'attr2'
    ]
)
# Display the DataFrame

,frame_id,track_id,x,y,width,height,confidence,class_id,visibility,attr1,attr2
0,3,1,201.25,813.94,117.51,263.89,0.90,-1,-1,-1,-1
1,3,2,350.66,812.14,110.69,267.01,0.89,-1,-1,-1,-1
2,3,3,1581.77,679.13,94.08,228.13,0.89,-1,-1,-1,-1
3,3,4,1655.66,836.58,97.71,242.05,0.89,-1,-1,-1,-1
4,3,5,1469.53,642.59,72.31,202.78,0.85,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...
11588,429,125,528.54,699.58,82.28,216.63,0.78,-1,-1,-1,-1
11589,429,131,1162.05,632.65,63.40,192.51,0.75,-1,-1,-1,-1
11590,429,132,1101.66,650.26,61.06,213.86,0.63,-1,-1,-1,-1
11591,429,154,1646.77,629.53,47.52,61.71,0.32,-1,-1,-1,-1
